In [31]:
import paramiko

In [32]:
jumpserver_private_addr = '10.251.6.31'   # The internal IP Address for the Jump server
local_IP_addr = '127.0.0.1' # IP Address of the machine you are connecting from

username = "adm.davies"
password = "!Lepsodizle7!"

In [33]:
def open_session(IP):
    print(f"Trying to establish a connection to: {IP}")
    jumpbox=paramiko.SSHClient()
    jumpbox.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    jumpbox.connect(jumpserver_private_addr, username=username, password=password )
    jumpbox_transport = jumpbox.get_transport()
    src_addr = (local_IP_addr, 22)
    dest_addr = (IP, 22)
    jumpbox_channel = jumpbox_transport.open_channel("direct-tcpip", dest_addr, src_addr)
    target=paramiko.SSHClient()
    target.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    target.connect(dest_addr, username=username, password=password, sock=jumpbox_channel)
    print(f"Connection to IP: {IP} established")
    return target, jumpbox, True

In [29]:
def interface_descr(IP, int_name):
    command = f"show run interface {int_name} | inc description"
    ssh, jumpbox, connection = open_session(IP)
    stdin, stdout, stderr = ssh.exec_command(command)
    stdout = stdout.read()
    stdout = stdout.decode("utf-8")
    int_description = stdout
    ssh.close()
    jumpbox.close()
    print(int_description)

In [30]:
interface_descr("10.112.255.1", "Po35")

Trying to establish a connection to: 10.112.255.1
Connection to IP: 10.112.255.1 established
 description ***Etherchannel to CISFS ISCi/NetApp CR1***



In [20]:
open_session("10.112.255.1")

Open Session Error: An unknown error occured for IP: 10.112.255.1!


(None, None, False)